# **Imports**:

In [21]:
import string
import numpy as np
import pandas as pd
import re
import nltk

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import nltk
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# MAIN

Load DataFrame from Phishing Emails:

In [22]:
phishingEmailDf = load_and_display_dataframe('sample_data/CEAS_08.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31573 entries, 0 to 31572
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    31573 non-null  object
 1   receiver  31192 non-null  object
 2   date      31573 non-null  object
 3   subject   31549 non-null  object
 4   body      31573 non-null  object
 5   label     31573 non-null  int64 
 6   urls      31573 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 1.7+ MB


,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1
...,...,...,...,...,...,...,...
31568,CNN Alerts <charlene-detecton@btcmarketing.com>,email1007@gvc.ceas-challenge.cc,"Fri, 08 Aug 2008 10:34:50 -0400",CNN Alerts: My Custom Alert,\n\nCNN Alerts: My Custom Alert\n\n\n\n\n\n\n ...,1,0
31569,CNN Alerts <idgetily1971@careplusnj.org>,email104@gvc.ceas-challenge.cc,"Fri, 08 Aug 2008 10:35:11 -0400",CNN Alerts: My Custom Alert,\n\nCNN Alerts: My Custom Alert\n\n\n\n\n\n\n ...,1,0
31570,Abhijit Vyas <xpojhbz@gmail.com>,fxgmqwjn@triptracker.net,"Fri, 08 Aug 2008 22:00:43 +0800",Slideshow viewer,Hello there ! \nGreat work on the slide show v...,0,0
31571,Joseph Brennan <vupzesm@columbia.edu>,zqoqi@spamassassin.apache.org,"Fri, 08 Aug 2008 09:00:46 -0500",Note on 2-digit years,"\nMail from sender , coming from intuit.com\ns...",0,0


Show Details

In [23]:
phishingEmailDf.info()
phishingEmailDf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31573 entries, 0 to 31572
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    31573 non-null  object
 1   receiver  31192 non-null  object
 2   date      31573 non-null  object
 3   subject   31549 non-null  object
 4   body      31573 non-null  object
 5   label     31573 non-null  int64 
 6   urls      31573 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 1.7+ MB


,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1
...,...,...,...,...,...,...,...
31568,CNN Alerts <charlene-detecton@btcmarketing.com>,email1007@gvc.ceas-challenge.cc,"Fri, 08 Aug 2008 10:34:50 -0400",CNN Alerts: My Custom Alert,\n\nCNN Alerts: My Custom Alert\n\n\n\n\n\n\n ...,1,0
31569,CNN Alerts <idgetily1971@careplusnj.org>,email104@gvc.ceas-challenge.cc,"Fri, 08 Aug 2008 10:35:11 -0400",CNN Alerts: My Custom Alert,\n\nCNN Alerts: My Custom Alert\n\n\n\n\n\n\n ...,1,0
31570,Abhijit Vyas <xpojhbz@gmail.com>,fxgmqwjn@triptracker.net,"Fri, 08 Aug 2008 22:00:43 +0800",Slideshow viewer,Hello there ! \nGreat work on the slide show v...,0,0
31571,Joseph Brennan <vupzesm@columbia.edu>,zqoqi@spamassassin.apache.org,"Fri, 08 Aug 2008 09:00:46 -0500",Note on 2-digit years,"\nMail from sender , coming from intuit.com\ns...",0,0


Pre-Processing

In [24]:
nltk.download('stopwords')
nltk.download('punkt_tab')


nltk_tokens = nltk.word_tokenize(phishingEmailDf.body.iloc[4])
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
nltk_cleaned = [token.lower() for token in nltk_tokens if token.lower() not in stop_words and not re.match(r"[^\w\s]", token)]
nltk_cleaned


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['welcomefastshippingcustomersupport', 'http']

In [25]:
# Function to clean and tokenize email body
def clean_email_body(body):
    nltk_tokens = nltk.word_tokenize(body)
    cleaned_tokens = [token.lower() for token in nltk_tokens if token.lower() not in stop_words and not re.match(r"[^\w\s]", token)]
    return cleaned_tokens

# Loop over the first 10 rows of the dataframe
cleaned_bodies = []
for index in range(10):
    body = phishingEmailDf['body'].iloc[index]
    cleaned_body = clean_email_body(body)
    cleaned_bodies.append(cleaned_body)

# You can now view the cleaned bodies of the first 10 emails
for index, cleaned_body in enumerate(cleaned_bodies):
    print(f"Email {index}: {cleaned_body}")

Email 0: ['buck', 'troubles', 'caused', 'small', 'dimension', 'soon', 'become', 'lover', 'woman', 'able', 'resist', 'http', 'come', 'even', 'nazi', 'tanks', 'rolling', 'streets', 'dreamersphilosopher', 'journalist', 'still', 'sure.i']
Email 1: ['upgrade', 'sex', 'pleasures', 'techniques', 'http']
Email 2: ['daily', 'top', '10', 'cnn.com', 'top', 'videos', 'stories', 'aug', '1', '2008', '3:58', 'pm', 'edt', 'top', '10', 'videos', '1', 'montauk', 'http', 'url=/video/us/2008/07/31/moos.montauk.monster.cnn', 'devil', 'dog', 'turtle', 'montauk', 'monster', 'cnn', 'jeanne', 'moos', 'asks', 'thing', '2', 'racy', 'photos', 'toddler', 'mom', 'http', 'url=/video/crime/2008/07/31/ng.racy.photos.cnn', '3', 'news', 'absurd', 'episode', '54', 'http', 'url=/video/podcasts/absurd/site/2008/08/01/nota.episode.54.cnn', '4', 'police', 'beating', 'dispute', 'http', 'url=/video/us/2008/07/31/levs.police.video.cnn', '5', 'mom', 'pleads', 'girl', 'return', 'http', 'url=/video/crime/2008/08/01/hill.boss.reigh

In [26]:
import nltk
import re

# Define stop words (this is just an example, ensure you have the actual stop words list)
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to clean and tokenize email body
def clean_email_body(body):
    # Tokenize the email body using NLTK
    nltk_tokens = nltk.word_tokenize(body)

    # Clean tokens:
    # - Convert to lowercase
    # - Remove stop words
    # - Remove tokens that are URLs, contain alphabetic sequences in order, or match example websites
    cleaned_tokens = [
        token.lower() for token in nltk_tokens
        if token.lower() not in stop_words and
        not re.match(r"[^\w\s]", token) and  # Remove non-alphanumeric characters (punctuation)
        not re.match(r'http[s]?://[^\s]+', token) and  # Remove URLs
        not re.match(r'url=[^\s]+', token) and  # Remove tokens with 'url=' pattern
        not re.match(r'[a-z]{2,}', token) and  # Remove alphabetic sequences in order (e.g., 'abc')
        not re.match(r'\b(cnn|bbc|nytimes|example)\.com\b', token)  # Remove example websites (expand as needed)
    ]
    return cleaned_tokens




In [27]:
# Apply the cleaning function to the 'body' column and create a new column for cleaned text
phishingEmailDf['cleaned_body'] = phishingEmailDf['body'].apply(clean_email_body)



In [28]:
# Initialize the TF-IDF Vectorizer to transform the text data into numerical features
vectorizer = TfidfVectorizer(max_features=1000)  # Limiting to 1000 features for simplicity
X = vectorizer.fit_transform(phishingEmailDf['cleaned_body']).toarray()

# Define the target variable (Assuming it is in 'target' column, modify as necessary)
y = phishingEmailDf['target']

# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the regression model (Linear Regression here)
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

AttributeError: 'list' object has no attribute 'lower'